## A simple model for demand and supply of publicly-provided services in a city

In [ ]:
from enum import Enum
import os.path
import json

import numpy as np
import pandas as pd
import geopandas as gpd
import geopy, geopy.distance
import shapely
from sklearn import gaussian_process

from matplotlib import pyplot as plt 
import seaborn as sns
plt.rcParams['figure.figsize']= (20,14)

In [ ]:
## TODO: find way to put this into some global settings
import os
import sys
nb_dir = os.path.dirname(os.getcwd())
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from references import common_cfg

In [ ]:
from src.models.city_items import AgeGroup, ServiceArea, ServiceType, SummaryNorm # enum classes for the model

In [ ]:
from references import istat_kpi

In [ ]:
from src.models.services_supply import ServiceUnit, ServiceValues, ServiceEvaluator, MappedPositionsFrame
from src.models.factories import UnitFactory, SchoolFactory, LibraryFactory 
from src.models.demand import DemandFrame
from src.models.process_tools import GridMaker, ValuesPlotter
from src.models.city_items import get_random_pos

In [ ]:
quicktest = [ServiceUnit(ServiceType.Library, 'Duomo', ageDiffusionIn=None), 
        ServiceUnit(ServiceType.Library, 'Ripamonti', 
                    position=geopy.Point(45.43, 9.201), ageDiffusionIn=None)]
ServiceEvaluator(quicktest).evaluate_services_at(MappedPositionsFrame(get_random_pos(4)))
del quicktest

In [ ]:
## Load scuole
scuoleFile =  '../data/processed/Milano_datiScuole.csv'
schoolLoader = UnitFactory.createLoader(ServiceType.School, scuoleFile)

# Initialise with a default lengthscale of 0.5 km
schoolUnits = schoolLoader.load(meanRadius=0.5)
schoolEval = ServiceEvaluator(schoolUnits)

In [ ]:
## Load biblioteche
bibliotecheFile =  '../data/processed/Milano_biblioteche.csv'
bibliotecheLoader = UnitFactory.createLoader(ServiceType.Library, bibliotecheFile)

# Initialise with a default lengthscale of 0.5 km
libraryUnits = bibliotecheLoader.load(meanRadius=0.5)

In [ ]:
# call grid making to discretise service evaluation, this is an alternative to evaluating on the demand units
milanoGridMK = GridMaker({'quartieri':'../data/raw/Milano_specific/Milano_quartieri.geojson'}, gridStep=.4)
testEvaluator = ServiceEvaluator(schoolUnits[:200])
# call evaluation on internal points only
valuesGrid = testEvaluator.evaluate_services_at(milanoGridMK.grid)
plotterNew = ValuesPlotter(valuesGrid)
plotterNew.plot_locations()
plotterNew.plot_service_levels(ServiceType.School) 

In [ ]:
class KPICalculator:
    '''Class to aggregate demand and evaluate section based and position based KPIs'''
    
    def __init__(self, demandFrame, serviceUnits, cityName):
        assert cityName in common_cfg.cityList, 'Unrecognized city name %s' % cityName
        assert isinstance(demandFrame, DemandFrame),'Demand frame expected'
        assert all([isinstance(su, ServiceUnit) for su in serviceUnits]),'Service units list expected'
        
        self.city = cityName
        self.demand = demandFrame
        self.sources = serviceUnits
        # initialise the service evaluator
        self.evaluator = ServiceEvaluator(serviceUnits)
        self.servicePositions = self.evaluator.servicePositions
        # initialise output values
        self.serviceValues = ServiceValues(self.demand.mappedPositions)
        self.weightedValues = ServiceValues(self.demand.mappedPositions)
        self.quartiereKPI = {}
        self.istatKPI = pd.DataFrame()

        # derive Ages frame
        ageMIndex = [demandFrame[common_cfg.IdQuartiereColName],
                         demandFrame[common_cfg.positionsCol].apply(tuple)]
        self.agesFrame = demandFrame[AgeGroup.all()].set_index(ageMIndex)
        self.agesTotals = self.agesFrame.groupby(level=0).sum()
        
        
    def evaluate_services_at_demand(self):
        self.serviceValues = self.evaluator.evaluate_services_at(
            self.demand.mappedPositions)
        return self.serviceValues
    
    
    def compute_kpi_for_localized_services(self):
        assert self.serviceValues, 'Service values not available, have you computed them?'
        # get mean service levels by quartiere, weighting according to the number of citizens
        for service, data in self.serviceValues.items():
            checkRange = {}
            for col in self.agesFrame.columns: # iterate over columns as Enums are not orderable...
                if col in service.demandAges:
                    self.weightedValues[service][col] = pd.Series.multiply(
                        data[col], self.agesFrame[col])
                else:
                    self.weightedValues[service][col] = np.nan*data[col]
            
            checkRange = (data.groupby(common_cfg.IdQuartiereColName).min()-np.finfo(float).eps,
                              data.groupby(common_cfg.IdQuartiereColName).max()+np.finfo(float).eps)
            
            # sum weighted fractions by neighbourhood
            weightedSums = self.weightedValues[service].groupby(common_cfg.IdQuartiereColName).sum()
            # set to NaN value the AgeGroups that have no people or there is no demand for the service
            weightedSums[self.agesTotals == 0] = np.nan
            weightedSums.iloc[:, ~weightedSums.columns.isin(service.demandAges)] = np.nan
            
            self.quartiereKPI[service] = (weightedSums/self.agesTotals).reindex(
                columns=AgeGroup.all(), copy=False)
            
            # check that the weighted mean lies between min and max in the neighbourhood
            for col in self.quartiereKPI[service].columns:
                bGood = (self.quartiereKPI[service][col].between(
                    checkRange[0][col], checkRange[1][col]) | self.quartiereKPI[service][col].isnull())
                assert all(bGood), 'Unexpected error in mean computation'
            
        return self.quartiereKPI

    def compute_kpi_for_istat_values(self):
        kpiFrame = istat_kpi.wrangle_istat_cpa2011(
            self.demand.groupby(common_cfg.IdQuartiereColName).sum(),
            self.city)
        self.istatKPI = kpiFrame
        return self.istatKPI

In [ ]:
frame = DemandFrame.create_from_istat_cpa('Milano')
demandTest = DemandFrame(frame.head(500).copy(), False)
tt = KPICalculator(demandTest, schoolUnits[::10], 'Milano')
kk = tt.compute_kpi_for_istat_values()
tt.compute_kpi_for_localized_services()

In [ ]:
from scipy.interpolate import griddata
                
class ValuesPlotter:
    '''
    A class that plots various types of output from ServiceValues
    '''
    def __init__(self, serviceValues, bOnGrid=False):
        assert isinstance(serviceValues, ServiceValues), 'ServiceValues class expected'
        self.values = serviceValues
        self.bOnGrid = bOnGrid
        
        
    def plot_locations(self):
        '''
        Plots the locations of the provided ServiceValues'
        '''
        coordNames = common_cfg.coordColNames
        plt.figure()
        plt.scatter(self.values.mappedPositions[coordNames[0]],
                    self.values.mappedPositions[coordNames[1]])
        plt.xlabel(coordNames[0])
        plt.ylabel(coordNames[1])
        plt.axis('equal')
        plt.show()
        return None
    
        
    def plot_service_levels(self, servType, gridDensity=40, nLevels=50):
        '''
        Plots a contour graph of the results for each ageGroup.
        '''
        assert isinstance(servType, ServiceType), 'ServiceType expected in input'
        
        for ageGroup in self.values[servType].keys():
            
            xPlot,yPlot,z = self.values.plot_output(servType, ageGroup)
            
            if np.count_nonzero(z) > 0:
                if self.bOnGrid:
                    gridShape = (len(set(xPlot)), len(set(yPlot.flatten())))
                    assert len(xPlot) == gridShape[0]*gridShape[1], 'X values do not seem on a grid'
                    assert len(yPlot) == gridShape[0]*gridShape[1], 'Y values do not seem on a grid'
                    xi = np.array(xPlot).reshape(gridShape)
                    yi = np.array(yPlot).reshape(gridShape)
                    zi = z.reshape(gridShape)
                else:
                    # grid the data using natural neighbour interpolation
                    xi = np.linspace(min(xPlot), max(xPlot), gridDensity)
                    yi = np.linspace(min(yPlot), max(yPlot), gridDensity)
                    zi = griddata((xPlot, yPlot), z, (xi[None,:], yi[:,None]), 'nearest')
                    
                plt.figure()
                plt.title(ageGroup)
                CS = plt.contourf(xi, yi, zi, nLevels)
                cbar = plt.colorbar(CS)
                cbar.ax.set_ylabel('Service level')
                plt.show()
            
        return None

In [ ]:
plotterDemand = ValuesPlotter(tt.weightedValues)
plotterDemand.plot_service_levels(ServiceType.School)
plotterDemandEval = ValuesPlotter(tt.serviceValues)
plotterDemandEval.plot_service_levels(ServiceType.School)

In [ ]:
plotterNew = ValuesPlotter(ServiceValues(ServiceEvaluator(schoolUnits[::10]).servicePositions))
plotterNew.plot_service_levels(ServiceType.School)
plotterNew.plot_locations()

In [ ]:
class JSONWriter:
    def __init__(self, kpiCalc):
        assert isinstance(kpiCalc, KPICalculator), 'KPI calculator is needed'
        self.layersData = kpiCalc.quartiereKPI
        self.istatData = kpiCalc.istatKPI
        self.city = kpiCalc.city
        self.areasTree = {}
        for s in self.layersData:
            area = s.serviceArea    
            self.areasTree[area] = [s] + self.areasTree.get(area, []) 

        
    def make_menu(self):
        jsonList = common_cfg.make_output_menu(self.city, 
            services=list(self.layersData.keys()))
        return jsonList
        
        
    def make_serviceareas_output(self, precision=4):
        out = dict()
        
        # make istat layer
        istatFrame = self.istatData.round(precision)
        origType = istatFrame.index.dtype.type
        dataIstat = istatFrame.reset_index().to_dict(orient='records')
        # restore type as pandas has a bug and casts to float if int
        for quartiereData in dataIstat:
            oldValue = quartiereData[common_cfg.IdQuartiereColName]
            if origType in (np.int32, np.int64, int):
                quartiereData[common_cfg.IdQuartiereColName] = int(oldValue)
        out[common_cfg.istatLayerName] = dataIstat
        
        # make layers
        for area, layers in self.areasTree.items():
            layerList = []
            for service in layers:
                data = self.layersData[service].round(precision)
                layerList.append(pd.Series(
                    data[AgeGroup.all()].as_matrix().tolist(),
                    index=data.index, name=service.name))
            areaData = pd.concat(layerList, axis=1).reset_index()
            print(areaData)
            out[area.value] = areaData.to_dict(orient='records')
        return out 
    
    
    def write_all_files_to_default_path(self):
        # build and write menu
        with open(os.path.join(
            '../',common_cfg.vizOutputPath, 'menu.js'), 'w') as menuFile:
            json.dump(self.make_menu(), menuFile, sort_keys=True,
                     indent=4, separators=(',', ' : '))
        
        # build and write all areas
        areasOutput = self.make_serviceareas_output()
        for name, data in areasOutput.items():
            filename =  '%s_%s.js'%(self.city,name)
            with open(os.path.join('../', common_cfg.outputPath,
                        filename), 'w') as areaFile:
                json.dump(data, areaFile, sort_keys=True,
                          indent=4, separators=(',', ' : '))
    
    

In [ ]:
#demandTest[common_cfg.IdQuartiereColName]
testCalc = KPICalculator(frame, schoolUnits[:10] + libraryUnits[:10], 'Milano')
testCalc.compute_kpi_for_istat_values()
testCalc.compute_kpi_for_localized_services()
yy = JSONWriter(testCalc)
yy.write_all_files_to_default_path()